In [ ]:
# Import data via Twitter's API to Elasticsearch using Kibana to view the results

import tweepy
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import sys
import json
import time
from textwrap import TextWrapper
from datetime import datetime
from elasticsearch import Elasticsearch

consumer_key = "CONSUMER_KEY"
consumer_secret = "CONSUMER_SECRET_KEY"
access_token = "ACCESS_TOKEN"
access_token_secret = "ACCESS_TOKEN_SECRET"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

ES_HOST = {"host" : "localhost", "port" : 9200}
es = Elasticsearch(hosts = [ES_HOST])


class StreamListener(tweepy.StreamListener):
    
        print('Starting StreamListener')
        status_wrapper = TextWrapper(width=60, initial_indent='    ', subsequent_indent='    ')

        def on_error(self, status):
                    print(status)
                    
        def on_status(self, status):
                try:
                        print ("n%s %s" % (status.author.screen_name, status.created_at))
                        json_data = status._json
                        print(json_data['text'])
                        print(json_data['id'])
                        es.index(index="twitter_trump",
                                  doc_type="twitter_tweet",
                                  id=json_data['id'],
                                  body=json_data,
                                  ignore=400
                                 )                       
                except Exception as e:
                        print(e)
                        pass
       
        
print('Starting Receiving')
runtime = 1 #this is in seconds

streamer = tweepy.Stream(auth=auth, listener=StreamListener(), timeout=30)

streamer.filter(track=['Trump'], is_async=True)        
#streamer.userstream(None) if you want to query a user


print("sleeping")
time.sleep(runtime)
print("waking")
streamer.disconnect()

print ('Ending program')

